# **뉴스 키워드 분석**

In [5]:
import numpy as np
import pandas as pd

### 데이터 로드

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
train_data = pd.read_csv('/content/drive/MyDrive/GBT Hackerton/train.csv')

In [25]:
train_data.head()

,ID,분류,제목,키워드,대분류
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...",문화
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트...",지역
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화...",지역
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특...",지역
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코...",국제


In [26]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
 4   대분류     54609 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


### 대분류별 분석

In [27]:
# 대분류 컬럼 생성
def create_main_category(category):
    if ':' in category:
        return category.split(':')[0]  # ':' 앞의 값 반환
    else:
        return category  # ':' 없으면 원래 값 반환

train_data['대분류'] = train_data['분류'].apply(create_main_category)
train_data['대분류'].unique()

array(['문화', '지역', '국제', '정치', '경제', '사회', '스포츠', 'IT_과학'], dtype=object)

In [28]:
train_data['대분류'].value_counts()

,count
대분류,
지역,26950
경제,10534
사회,8245
정치,2521
문화,2500
스포츠,2035
IT_과학,1487
국제,337


In [31]:
### 지역
filtered_df = train_data[train_data['대분류'] == '지역']

# 각 행의 키워드 개수 계산
filtered_df['키워드 개수'] = filtered_df['키워드'].apply(lambda x: len(x.split(',')))

# 평균, 최대 개수 계산
mean_count = filtered_df['키워드 개수'].mean()
max_count = filtered_df['키워드 개수'].max()

# 결과 출력
print(f'키워드 평균 개수: {mean_count}')
print(f'키워드 최대 개수: {max_count}')

키워드 평균 개수: 163.33276437847866
키워드 최대 개수: 2581


<ipython-input-31-ac18d6a33eb9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['키워드 개수'] = filtered_df['키워드'].apply(lambda x: len(x.split(',')))


In [37]:
# 각 대분류별로 키워드를 리스트로 모으기
keywords_by_category = train_data.groupby('대분류')['키워드'].apply(lambda x: ','.join(x)).str.split(',')

# 모든 키워드를 하나의 리스트로 결합
all_keywords = keywords_by_category.explode()

# 각 키워드의 개수 계산
keyword_counts = all_keywords.value_counts()

# 개수가 1인 키워드 필터링
single_keyword_counts = keyword_counts[keyword_counts == 1].index

# 대분류별 개수가 1개인 키워드 개수 및 총 개수 계산
result = keywords_by_category.apply(lambda x: {
    '총 개수': len(set(x)),
    '개수가 1개인 키워드 개수': len(set(x) & set(single_keyword_counts))
})

# 결과 출력
print('대분류별 키워드 총 개수와 개수가 1개인 키워드 개수:')
print(result)

대분류별 키워드 총 개수와 개수가 1개인 키워드 개수:
대분류
IT_과학      {'총 개수': 26207, '개수가 1개인 키워드 개수': 3447}
경제        {'총 개수': 95207, '개수가 1개인 키워드 개수': 25059}
국제          {'총 개수': 10226, '개수가 1개인 키워드 개수': 939}
문화        {'총 개수': 50054, '개수가 1개인 키워드 개수': 12502}
사회        {'총 개수': 68450, '개수가 1개인 키워드 개수': 14404}
스포츠        {'총 개수': 21515, '개수가 1개인 키워드 개수': 4303}
정치         {'총 개수': 43251, '개수가 1개인 키워드 개수': 8568}
지역       {'총 개수': 141926, '개수가 1개인 키워드 개수': 39936}
Name: 키워드, dtype: object


In [39]:
# 각 대분류별로 키워드를 리스트로 모으기
keywords_by_category = train_data.groupby('대분류')['키워드'].apply(lambda x: ','.join(x)).str.split(',')

# 대분류별로 키워드를 개별적으로 나누기
exploded_keywords = keywords_by_category.explode()

# 대분류별로 키워드 개수 계산
top_keywords_by_category = exploded_keywords.groupby('대분류').value_counts().groupby(level=0).head(50)

# 결과 출력
print('각 대분류별 상위 50개 키워드:')
for category, group in top_keywords_by_category.groupby(level=0):
    print(f'\n대분류: {category}')
    print(group)

각 대분류별 상위 50개 키워드:

대분류: IT_과학
대분류    키워드 
IT_과학  서비스     3329
       용인시     3195
       제공      1402
       운영      1203
       용인      1191
       기업      1142
       활용      1118
       정보      1090
       지원      1089
       진행      1064
       이용      1044
       사업      1020
       교육       963
       시장       956
       도서관      915
       지역       898
       센터       840
       데이터      818
       대상       809
       AI       800
       로봇       788
       시민들      784
       관계자      771
       구축       759
       시민       752
       분야       741
       확인       707
       가능       678
       계획       658
       온라인      651
       플랫폼      644
       경기       608
       홈페이지     605
       확대       603
       사용       601
       경기도      572
       스마트      570
       시작       569
       프로그램     568
       참여       551
       신청       548
       설치       548
       대표       546
       캐릭터      541
       콘텐츠      509
       디지털      500
       마련       490
       조아용      4

In [43]:
# 동일한 키워드가 다른 대분류에서도 존재하는 경우 찾기
duplicate_keywords = top_keywords_by_category[top_keywords_by_category.index.get_level_values(1).duplicated(keep=False)]

# 결과를 새로운 리스트에 담기
new_list = duplicate_keywords.index.get_level_values(1).unique().tolist()

# 결과 출력
print('동일한 키워드가 다른 대분류의 상위 키워드에 존재하는 경우:')
new_list

동일한 키워드가 다른 대분류의 상위 키워드에 존재하는 경우:


['용인시',
 '제공',
 '운영',
 '용인',
 '기업',
 '지원',
 '진행',
 '이용',
 '사업',
 '교육',
 '시장',
 '도서관',
 '지역',
 '센터',
 '대상',
 '시민들',
 '관계자',
 '시민',
 '확인',
 '가능',
 '계획',
 '경기',
 '경기도',
 '시작',
 '프로그램',
 '참여',
 '신청',
 '설치',
 '대표',
 '마련',
 '예정',
 '반도체',
 '단지',
 '산업',
 '조성',
 '서울',
 '투자',
 '정부',
 '도시',
 '추진',
 '기준',
 '처인구',
 '전국',
 '에버랜드',
 '한국',
 '혐의',
 '경찰',
 '발생',
 '조사',
 '모습',
 '이날',
 '사고',
 'A씨',
 '상황',
 '세계',
 '사건',
 '사진',
 '이상일',
 '경제',
 '문화',
 '가족',
 '생각',
 '환경',
 '시설',
 '특례시']